<a href="https://colab.research.google.com/github/rahulrajpr/AdvancedHyperParameterOptimisation/blob/Optuna/Optuna_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Optuna Search**

- Ransom Search
- TPE Search
- CM ES Search
- Grid Search

**Optuna is compateble for optimisig the hyperparmeters of all kind of machine learning packages like sklearn, keras, tensorflow, pytorch etc**

In optuna, the hyperparameter space is defined inside the objective function

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.datasets import load_breast_cancer

In [3]:
dataset = load_breast_cancer()
data = dataset.data
target = dataset.target
columns = dataset.feature_names
df = pd.DataFrame(data = data, columns = columns)
df['target'] = target
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

In [4]:
X = df.drop('target', axis =1)
y = df['target']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train,X_test,y_train, y_test = train_test_split(X,y,train_size = 0.7, random_state = 100, stratify = y)

Define the objective function

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
from sklearn.model_selection import cross_val_score

In [12]:
def objective(trial):

  # defing the hyper parameter space inside the objective function, which is good thing in optuna

  n_estimators = trial.suggest_int('n_estimators',100,1000)
  criterion = trial.suggest_categorical('criterion',['gini','entropy'])
  max_depth = trial.suggest_int('max_depth',1,5)
  min_samples_split = trial.suggest_float('min_samples_split',0.01,1)

  model = RandomForestClassifier(n_estimators = n_estimators,
                                criterion = criterion,
                                max_depth = max_depth,
                                min_samples_split = min_samples_split)
  
  scores = cross_val_score(estimator = model,
                          X = X_train,
                          y = y_train,
                          cv = 5,
                          scoring = 'accuracy',
                          n_jobs = -1)

  mean_accuracy = np.mean(scores)

  return mean_accuracy # in optuna, you can choose the direction of scoring while defining the study, you can retun the metric here as it is.

**Random Search**

In [17]:
from IPython.core.display import clear_output
!pip install optuna
clear_output()

In [18]:
import optuna

In [19]:
# creating the study object
study = optuna.create_study(direction = 'maximize',
                            sampler = optuna.samplers.RandomSampler())

[I 2023-01-04 05:35:03,695] A new study created in memory with name: no-name-cd269785-26e7-4858-9329-c2dbf187e830


In [21]:
# optimize the study object to search through the hyperparameters

study.optimize(func = objective,
               n_trials = 50, 
               n_jobs =-1,
               show_progress_bar = True)

  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-01-04 05:37:59,297] Trial 1 finished with value: 0.6281645569620252 and parameters: {'n_estimators': 977, 'criterion': 'entropy', 'max_depth': 1, 'min_samples_split': 0.7034592012837281}. Best is trial 1 with value: 0.6281645569620252.
[I 2023-01-04 05:37:59,844] Trial 0 finished with value: 0.9120253164556962 and parameters: {'n_estimators': 983, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_split': 0.4760680924116854}. Best is trial 0 with value: 0.9120253164556962.
[I 2023-01-04 05:38:00,928] Trial 2 finished with value: 0.6281645569620252 and parameters: {'n_estimators': 231, 'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 0.8852810509008352}. Best is trial 0 with value: 0.9120253164556962.
[I 2023-01-04 05:38:10,144] Trial 3 finished with value: 0.6281645569620252 and parameters: {'n_estimators': 500, 'criterion': 'gini', 'max_depth': 2, 'min_samples_split': 0.8291747238189285}. Best is trial 0 with value: 0.9120253164556962.
[I 2023-01-04 05:38:12,043] Tr

Explore the study object

In [25]:
# best parameters
best_params = study.best_params
best_params

{'n_estimators': 351,
 'criterion': 'gini',
 'max_depth': 5,
 'min_samples_split': 0.019369243231351176}

In [23]:
# best metric
study.best_value

0.949746835443038

In [26]:
res = study.trials_dataframe()
res.head()

,number,value,datetime_start,datetime_complete,duration,params_criterion,params_max_depth,params_min_samples_split,params_n_estimators,state
0,0,0.912025,2023-01-04 05:37:44.685769,2023-01-04 05:37:59.844047,0 days 00:00:15.158278,entropy,4,0.476068,983,COMPLETE
1,1,0.628165,2023-01-04 05:37:44.686868,2023-01-04 05:37:59.296702,0 days 00:00:14.609834,entropy,1,0.703459,977,COMPLETE
2,2,0.628165,2023-01-04 05:37:59.304108,2023-01-04 05:38:00.927754,0 days 00:00:01.623646,gini,5,0.885281,231,COMPLETE
3,3,0.628165,2023-01-04 05:37:59.861566,2023-01-04 05:38:10.144112,0 days 00:00:10.282546,gini,2,0.829175,500,COMPLETE
4,4,0.937215,2023-01-04 05:38:00.935401,2023-01-04 05:38:12.042628,0 days 00:00:11.107227,entropy,5,0.232074,982,COMPLETE


In [27]:
res.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 10 columns):
 #   Column                    Non-Null Count  Dtype          
---  ------                    --------------  -----          
 0   number                    50 non-null     int64          
 1   value                     50 non-null     float64        
 2   datetime_start            50 non-null     datetime64[ns] 
 3   datetime_complete         50 non-null     datetime64[ns] 
 4   duration                  50 non-null     timedelta64[ns]
 5   params_criterion          50 non-null     object         
 6   params_max_depth          50 non-null     int64          
 7   params_min_samples_split  50 non-null     float64        
 8   params_n_estimators       50 non-null     int64          
 9   state                     50 non-null     object         
dtypes: datetime64[ns](2), float64(2), int64(3), object(2), timedelta64[ns](1)
memory usage: 4.0+ KB


In [32]:
sel_cols = ['params_n_estimators','params_max_depth','params_min_samples_split','params_criterion','value']

In [35]:
res = res[sel_cols].sort_values(by = 'value', ascending = False).reset_index()
res.head()

,index,params_n_estimators,params_max_depth,params_min_samples_split,params_criterion,value
0,35,841,5,0.062412,gini,0.949747
1,7,351,5,0.019369,gini,0.949747
2,15,311,3,0.067830,gini,0.947215
3,36,108,3,0.067437,entropy,0.944747
4,43,182,3,0.084881,entropy,0.944747


Lest create the best model & Evaluate

In [41]:
rfc_random = RandomForestClassifier(**best_params)
rfc_random.fit(X_train,y_train)

RandomForestClassifier(max_depth=5, min_samples_split=0.019369243231351176,
                       n_estimators=351)

In [37]:
from sklearn.metrics import accuracy_score

In [39]:
def evaluate_model(model):
  print('train accuracy : ',accuracy_score(y_train,model.predict(X_train)))
  print('test accuracy : ',accuracy_score(y_test,model.predict(X_test)))

In [42]:
evaluate_model(rfc_random)

train accuracy :  0.9949748743718593
test accuracy :  0.9766081871345029


**TPE Search**

In [48]:
study_tpe = optuna.create_study(direction = 'maximize',
                                sampler = optuna.samplers.TPESampler()) # TPE sampling search method

[I 2023-01-04 06:03:32,016] A new study created in memory with name: no-name-15b9a4e6-288c-468b-ac50-1373388d9473


In [49]:
study_tpe.optimize(func = objective,
               n_trials = 50, 
               n_jobs =-1,
               show_progress_bar = True)

  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-01-04 06:03:42,737] Trial 0 finished with value: 0.914493670886076 and parameters: {'n_estimators': 271, 'criterion': 'entropy', 'max_depth': 1, 'min_samples_split': 0.38870229308531834}. Best is trial 0 with value: 0.914493670886076.
[I 2023-01-04 06:03:44,284] Trial 1 finished with value: 0.9447151898734176 and parameters: {'n_estimators': 699, 'criterion': 'gini', 'max_depth': 4, 'min_samples_split': 0.14503613524666742}. Best is trial 1 with value: 0.9447151898734176.
[I 2023-01-04 06:03:45,945] Trial 2 finished with value: 0.9145569620253164 and parameters: {'n_estimators': 269, 'criterion': 'entropy', 'max_depth': 1, 'min_samples_split': 0.41461179206540094}. Best is trial 1 with value: 0.9447151898734176.
[I 2023-01-04 06:03:46,751] Trial 3 finished with value: 0.911993670886076 and parameters: {'n_estimators': 144, 'criterion': 'gini', 'max_depth': 1, 'min_samples_split': 0.4679297019422337}. Best is trial 1 with value: 0.9447151898734176.
[I 2023-01-04 06:03:53,541] Tr

In [51]:
best_params = study_tpe.best_params
best_params

{'n_estimators': 843,
 'criterion': 'gini',
 'max_depth': 5,
 'min_samples_split': 0.02616555617747296}

In [52]:
study_tpe.best_value

0.9522468354430378

In [53]:
rfc_tpe = RandomForestClassifier(**best_params)

In [54]:
rfc_tpe.fit(X_train, y_train)

RandomForestClassifier(max_depth=5, min_samples_split=0.02616555617747296,
                       n_estimators=843)

In [55]:
evaluate_model(rfc_tpe)

train accuracy :  0.992462311557789
test accuracy :  0.9707602339181286


**CMA-ES Search**

In [58]:
study_cmes = optuna.create_study(direction = 'maximize',
                                 sampler = optuna.samplers.CmaEsSampler()) # CM ES Sampler

[I 2023-01-04 06:14:22,341] A new study created in memory with name: no-name-897da36c-4408-4e54-8777-5c343eb8f1e7


In [59]:
study_cmes.optimize(func = objective,
                    n_trials = 50,
                    n_jobs = -1,
                    show_progress_bar = True)

  0%|          | 0/50 [00:00<?, ?it/s]

[I 2023-01-04 06:14:36,637] Trial 0 finished with value: 0.9472468354430379 and parameters: {'n_estimators': 254, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_split': 0.06603350943258746}. Best is trial 0 with value: 0.9472468354430379.
[W 2023-01-04 06:14:36,677] The parameter 'criterion' in trial#2 is sampled independently by using `RandomSampler` instead of `CmaEsSampler` (optimization performance may be degraded). `CmaEsSampler` does not support dynamic search space or `CategoricalDistribution`. You can suppress this warning by setting `warn_independent_sampling` to `False` in the constructor of `CmaEsSampler`, if this independent sampling is intended behavior.
[I 2023-01-04 06:14:38,315] Trial 1 finished with value: 0.9396835443037975 and parameters: {'n_estimators': 556, 'criterion': 'entropy', 'max_depth': 4, 'min_samples_split': 0.21427267813839582}. Best is trial 0 with value: 0.9472468354430379.
[W 2023-01-04 06:14:38,333] The parameter 'criterion' in trial#3 is sampl

In [60]:
study_cmes.best_value

0.9497784810126582

In [63]:
best_params = study_cmes.best_params
best_params

{'n_estimators': 550,
 'criterion': 'gini',
 'max_depth': 3,
 'min_samples_split': 0.19647059853196552}

In [64]:
rfc_cmes = RandomForestClassifier(**best_params)

In [65]:
rfc_cmes.fit(X_train,y_train)

RandomForestClassifier(max_depth=3, min_samples_split=0.19647059853196552,
                       n_estimators=550)

In [67]:
evaluate_model(rfc_cmes)

train accuracy :  0.964824120603015
test accuracy :  0.9649122807017544


**Grid Search Using Optuna**

In [68]:
# create a grid samples space

params_grid2 = {'n_estimators': [100,200,500,800,1000],
                'criterion': ['gini','entropy'],
                'max_depth':[1,2,3,5,8],
                'min_samples_split': [0.1,1.0]}

In [72]:
study_grid = optuna.create_study(direction = 'maximize',
                                 sampler = optuna.samplers.GridSampler(params_grid2)) # grid search have take one - argument - which are the grid of the hyperparameters

[I 2023-01-04 06:25:56,799] A new study created in memory with name: no-name-9208aae6-bcd1-42f0-b694-96df6f8acb70


In [73]:
study_grid.optimize(func = objective, 
                    n_jobs = -1,
                    show_progress_bar = True)

[I 2023-01-04 06:27:10,027] Trial 1 finished with value: 0.911993670886076 and parameters: {'n_estimators': 100, 'criterion': 'gini', 'max_depth': 1, 'min_samples_split': 0.1}. Best is trial 1 with value: 0.911993670886076.
[I 2023-01-04 06:27:11,336] Trial 2 finished with value: 0.6281645569620252 and parameters: {'n_estimators': 100, 'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 1.0}. Best is trial 1 with value: 0.911993670886076.
[I 2023-01-04 06:27:12,401] Trial 0 finished with value: 0.6281645569620252 and parameters: {'n_estimators': 1000, 'criterion': 'gini', 'max_depth': 8, 'min_samples_split': 1.0}. Best is trial 1 with value: 0.911993670886076.
[I 2023-01-04 06:27:18,362] Trial 4 finished with value: 0.9447151898734176 and parameters: {'n_estimators': 100, 'criterion': 'gini', 'max_depth': 5, 'min_samples_split': 0.1}. Best is trial 4 with value: 0.9447151898734176.
[I 2023-01-04 06:27:19,716] Trial 5 finished with value: 0.6281645569620252 and parameters: {'n_est

In [74]:
best_params = study_grid.best_params 
best_params

{'n_estimators': 500,
 'criterion': 'gini',
 'max_depth': 8,
 'min_samples_split': 0.1}

In [75]:
study_grid.best_value

0.949746835443038

In [76]:
rfc_grid = RandomForestClassifier(**best_params)

In [78]:
rfc_grid.fit(X_train,y_train)

RandomForestClassifier(max_depth=8, min_samples_split=0.1, n_estimators=500)

In [79]:
evaluate_model(rfc_grid)

train accuracy :  0.9723618090452262
test accuracy :  0.9649122807017544
